# Part A: Merging data sources and preparing data for analysis

#### Data sourced from the following two datasets from Kaggle:
- https://www.kaggle.com/usdot/flight-delays
- https://www.kaggle.com/selfishgene/historical-hourly-weather-data

In [ ]:
RUN_ON_COLAB = True

if RUN_ON_COLAB:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)

# file locations (change this if you want to run local)
airports_csv_path = "/content/drive/My Drive/UWaterloo Stats Course Group 2 Winter 2021/files_for_part_A/airports.csv" if RUN_ON_COLAB else "airports.csv"
flights_csv_path = "/content/drive/My Drive/UWaterloo Stats Course Group 2 Winter 2021/files_for_part_A/flights.csv" if RUN_ON_COLAB else "flights.csv"
humidity_csv_path = "/content/drive/My Drive/UWaterloo Stats Course Group 2 Winter 2021/files_for_part_A/humidity.csv" if RUN_ON_COLAB else "humidity.csv"
pressure_csv_path = "/content/drive/My Drive/UWaterloo Stats Course Group 2 Winter 2021/files_for_part_A/pressure.csv" if RUN_ON_COLAB else "pressure.csv"
wind_direction_csv_path = "/content/drive/My Drive/UWaterloo Stats Course Group 2 Winter 2021/files_for_part_A/wind_direction.csv" if RUN_ON_COLAB else "wind_direction.csv"
wind_speed_csv_path = "/content/drive/My Drive/UWaterloo Stats Course Group 2 Winter 2021/files_for_part_A/wind_speed.csv" if RUN_ON_COLAB else "wind_speed.csv"
temperature_csv_path = "/content/drive/My Drive/UWaterloo Stats Course Group 2 Winter 2021/files_for_part_A/temperature.csv" if RUN_ON_COLAB else "temperature.csv"
weather_description_csv_path = "/content/drive/My Drive/UWaterloo Stats Course Group 2 Winter 2021/files_for_part_A/weather_description.csv" if RUN_ON_COLAB else "weather_description.csv"

Mounted at /content/drive


### A.1 Creating the Aviation Dataframe (Cleaning and Merging flights.csv + airports.csv)

In [ ]:
import pandas as pd
import numpy as np
from dateutil import parser
from datetime import datetime

MAX_ROWS = None

def create_flight_datetime(row):
    time = row['SCHEDULED_DEPARTURE'] # ex 2350 => 23h50
    hour = time // 100
    minute = time % 100
  
    timestamp = datetime(row['YEAR'], row['MONTH'], row['DAY'], hour, minute)
    return timestamp

# read in the flights dataset using only the columns we deem relevant
COL_LIST = ['YEAR', 'MONTH', 'DAY', 'SCHEDULED_DEPARTURE', 'ORIGIN_AIRPORT', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'TAXI_OUT', 'WEATHER_DELAY', 'LATE_AIRCRAFT_DELAY', 'AIRLINE_DELAY', 'SECURITY_DELAY', 'AIR_SYSTEM_DELAY']
flights_df = pd.read_csv(flights_csv_path, usecols=COL_LIST, nrows=MAX_ROWS)

# fill delay columns with zeroes if no values are supplied
delay_columns = ['WEATHER_DELAY', 'LATE_AIRCRAFT_DELAY', 'AIRLINE_DELAY', 'SECURITY_DELAY', 'AIR_SYSTEM_DELAY']
flights_df[delay_columns] = flights_df[delay_columns].fillna(0) 

# transform the YEAR, MONTH, DATE and SCHEDULED DEPARTURE columns into a python datetime object
flights_df['SCHEDULED_DEPARTURE_DATETIME'] = flights_df.apply(create_flight_datetime, axis=1)

# drop those columns that are no longer being used
COLS_TO_DROP = ['YEAR', 'MONTH', 'DAY', 'SCHEDULED_DEPARTURE']
flights_df = flights_df.drop(COLS_TO_DROP, axis=1)

# load list of airports data
COL_LIST = ['IATA_CODE', 'CITY']
airports_df = pd.read_csv(airports_csv_path, usecols=COL_LIST)

# merge flights and airports dataframes
aviation_df = pd.merge(airports_df, flights_df, left_on='IATA_CODE', right_on='ORIGIN_AIRPORT')
COLS_TO_DROP = ['IATA_CODE', 'ORIGIN_AIRPORT']
aviation_df = aviation_df.drop(COLS_TO_DROP, axis=1)
aviation_df = aviation_df.rename(columns={'CITY': 'ORIGIN_CITY'})

aviation_df.head(3)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ORIGIN_CITY,DEPARTURE_DELAY,TAXI_OUT,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,SCHEDULED_DEPARTURE_DATETIME
0,Allentown,-8.0,12.0,-20.0,0,0,0.0,0.0,0.0,0.0,0.0,2015-01-01 06:00:00
1,Allentown,-7.0,13.0,-20.0,0,0,0.0,0.0,0.0,0.0,0.0,2015-01-01 06:35:00
2,Allentown,0.0,9.0,-21.0,0,0,0.0,0.0,0.0,0.0,0.0,2015-01-01 16:00:00


### A.2 Loading all of the weather datasets (humidity, temperature, description, etc...)

In [ ]:
def create_weather_datetime(row):
    return datetime.strptime(row['datetime'], '%Y-%m-%d %H:%M:%S')  

def weather_dataframe(filepath):
    # read in weather csv
    weather_df = pd.read_csv(filepath)

    # convert date value which is a string into python datetime
    weather_df['WEATHER_DATETIME'] = weather_df.apply(create_weather_datetime, axis=1)

    # drop the old datetime column along with cities that are not in the USA
    COLS_TO_DROP = ['datetime', 'Vancouver', 'Toronto', 'Montreal', 'Beersheba', 'Tel Aviv District', 'Eilat', 'Haifa', 'Nahariyya', 'Jerusalem']
    weather_df = weather_df.drop(COLS_TO_DROP, axis=1)

    # the aviation data is only for 2015, therefore we should truncate our weather dataframe
    # and only take the data from 2015
    mask_for_2015 = [weather_df['WEATHER_DATETIME'][i].year == 2015 for i in range(len(weather_df))]
    return weather_df[np.array(mask_for_2015, dtype=bool)]

In [ ]:
# grab all of the weather data as separate dataframes
weather_description_df = weather_dataframe(weather_description_csv_path)
temperature_df = weather_dataframe(temperature_csv_path)
wind_direction_df = weather_dataframe(wind_direction_csv_path)
wind_speed_df = weather_dataframe(wind_speed_csv_path)
humidity_df = weather_dataframe(humidity_csv_path)
pressure_df = weather_dataframe(pressure_csv_path)

In [ ]:
# we only want the airports for which entries exist in both flight and weather datasets (all weather datasets have same cities)
mask_for_airports_only_in_weather_df = [aviation_df['ORIGIN_CITY'][i] in weather_description_df.columns for i in range(len(aviation_df))]
aviation_df = aviation_df[np.array(mask_for_airports_only_in_weather_df, dtype=bool)]

aviation_df.head(3)

,ORIGIN_CITY,DEPARTURE_DELAY,TAXI_OUT,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,SCHEDULED_DEPARTURE_DATETIME
4603,Albuquerque,7.0,29.0,22.0,0,0,15.0,0.0,7.0,0.0,0.0,2015-01-01 06:00:00
4604,Albuquerque,0.0,24.0,3.0,0,0,0.0,0.0,0.0,0.0,0.0,2015-01-01 06:55:00
4605,Albuquerque,17.0,21.0,35.0,0,0,18.0,0.0,17.0,0.0,0.0,2015-01-01 06:55:00


### A.3 Merging the aviation and weather datasets one by one using a loop with `pd.merge_asof`

In [ ]:
def merge_dataframe_onto_aviation_df(left_df, weather_df, column_heading):
    # when using merge_asof, the join keys must be sorted
    left_df = left_df.sort_values('SCHEDULED_DEPARTURE_DATETIME')
    weather_df = weather_df.sort_values('WEATHER_DATETIME')

    # use merge_asof on the two sorted datasets, joining on date columns
    # since we have hourly weather samples, the merge direction should be 'nearest',
    # which means the flight time is rounded up or down to match nearest weather time,
    # within a tolerance of 10 minutes (since the weather can change a lot beyond that time)
    merged_df = pd.merge_asof(left_df, weather_df,
                  left_on='SCHEDULED_DEPARTURE_DATETIME',
                  right_on='WEATHER_DATETIME', direction='nearest',
                  tolerance=pd.Timedelta('10 minutes'))

    # match city with weather info and store it as the value of column_heading, then drop redundant columns
    merged_df[column_heading] = merged_df.apply(lambda row : row.loc[row['ORIGIN_CITY']], axis=1)
    COLS_TO_DROP = ['WEATHER_DATETIME', 'Portland', 'San Francisco', 'Seattle', 'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque', 'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City', 'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis', 'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami', 'Pittsburgh', 'Philadelphia', 'New York', 'Boston']
    return merged_df.drop(COLS_TO_DROP, axis=1).dropna()

In [ ]:
# certainly not the most elegant way to merge various datasets - this could be optimized
df1 = merge_dataframe_onto_aviation_df(aviation_df, temperature_df, 'TEMPERATURE')
df2 = merge_dataframe_onto_aviation_df(df1, humidity_df, 'HUMIDITY')
df3 = merge_dataframe_onto_aviation_df(df2, pressure_df, 'PRESSURE')
df4 = merge_dataframe_onto_aviation_df(df3, weather_description_df, 'WEATHER_DESCRIPTION')
df5 = merge_dataframe_onto_aviation_df(df4, wind_speed_df, 'WIND_SPEED')
df_final = merge_dataframe_onto_aviation_df(df5, wind_direction_df, 'WIND_DIRECTION')

df_final.head(3)

,ORIGIN_CITY,DEPARTURE_DELAY,TAXI_OUT,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,SCHEDULED_DEPARTURE_DATETIME,TEMPERATURE,HUMIDITY,PRESSURE,WEATHER_DESCRIPTION,WIND_SPEED,WIND_DIRECTION
0,Los Angeles,-8.0,12.0,-9.0,0,0,0.0,0.0,0.0,0.0,0.0,2015-01-01 00:10:00,276.9885,39.0,1030.0,moderate rain,2.0,10.0
1,Portland,-5.0,9.0,-12.0,0,0,0.0,0.0,0.0,0.0,0.0,2015-01-01 00:50:00,270.4600,60.0,1045.0,sky is clear,2.0,68.0
2,Las Vegas,25.0,11.0,6.0,0,0,0.0,0.0,0.0,0.0,0.0,2015-01-01 00:55:00,274.6850,49.0,1028.0,light snow,3.0,242.0


In [ ]:
# output to csv
df_final.to_csv('out.csv', index=False)

if RUN_ON_COLAB:
  !cp out.csv "drive/My Drive/UWaterloo Stats Course Group 2 Winter 2021/output_for_part_A/out.csv"

### A.4 Run tests to ensure that merged dataset is accurate

In [ ]:
def run_test(test_date, test_city, test_column, test_csv_path):
  test_weather_df = pd.read_csv(test_csv_path)
  value_from_weather_df = test_weather_df[  test_weather_df['datetime'] == test_date  ][test_city].iloc[0]
  value_from_df_final = df_final[
          (df_final['SCHEDULED_DEPARTURE_DATETIME'] == datetime.strptime(test_date, '%Y-%m-%d %H:%M:%S')) &
          (df_final['ORIGIN_CITY'] == test_city)
  ][test_column].iloc[0]
  return value_from_weather_df == value_from_df_final

TEST_PARAMS = [
  {
      'test_date': '2015-01-01 08:00:00',
      'test_city': 'Houston',
      'test_column': 'WIND_SPEED',
      'test_csv_path': wind_speed_csv_path
  },
  {
      'test_date': '2015-05-08 13:00:00',
      'test_city': 'Los Angeles',
      'test_column': 'HUMIDITY',
      'test_csv_path': humidity_csv_path
  },
  {
      'test_date': '2015-12-31 19:00:00',
      'test_city': 'San Francisco',
      'test_column': 'WEATHER_DESCRIPTION',
      'test_csv_path': weather_description_csv_path
  }
]

from operator import itemgetter

for test in TEST_PARAMS:
  test_date, test_city, test_column, test_csv_path = itemgetter('test_date', 'test_city', 'test_column', 'test_csv_path')(test)
  print(run_test(test_date, test_city, test_column, test_csv_path)) # Should print out all True

True
True
True
